In [83]:
!pip3 install requests
!pip3 install python-dotenv
!pip3 install tqdm


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [88]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env
API_KEY = os.getenv("NYT_API_KEY")
print(API_KEY)


InVLUg7OVJNpNIvbcJ1xTMAmw7COxUjh


In [85]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm 
import time

In [89]:
startDate = datetime(2016, 1, 1)
endDate = datetime(2016,1,1)
articlesData = []
endpoint = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
params = {
          # "q": "economy OR trade OR Federal Reserve OR policy OR Canada OR politics",
          "fq": 'subsection_name:("Politics" OR "Financial" OR "Your Money")',
          "begin_date": startDate.strftime("%Y%m%d"),
          "end_date": endDate.strftime("%Y%m%d"),
          "sort": "relevance", 
          "api-key": "MDhBwmCEr3FfhRNn63ogkPOowV3bklnY"
        }

response = requests.get(endpoint, params=params)
docs = response.json().get('response', {}).get('docs', [])
for doc in docs:
                article_info = {
                    "title": doc.get("headline", {}).get("main", ""),
                    "snippet": doc.get("snippet", ""),
                    "url": doc.get("web_url", ""),
                    "published_date": doc.get("pub_date", ""),
                    "section": doc.get("section_name", ""),
                    "subsection": doc.get("subsection_name", ""),
                }
                print(article_info)
                articlesData.append(article_info)

articledf = pd.DataFrame(articlesData)
print(articledf)
if response.status_code == 200:
      print("yay")
    
else:
  print(f"Error: {response.status_code} - {response.text}")

{'title': 'Hillary Clinton Raised $37 Million in Last 3 Months', 'snippet': 'Hillary Clinton’s presidential campaign raised $37 million in the three-month period that ended Dec. 31, bringing her total war chest for use in the Democratic presidential contest to more than $112 million.', 'url': 'https://www.nytimes.com/politics/first-draft/2016/01/01/hillary-clinton-raised-37-million-in-last-3-months/', 'published_date': '2016-01-01T19:07:09+0000', 'section': 'U.S.', 'subsection': 'Politics'}
{'title': 'How Jeb Bush Hopes to Save His Candidacy', 'snippet': 'Rival campaigns seem to be counting him out, but Mr. Bush’s allies have a long-shot plan they hope will win him the Republican nomination.', 'url': 'https://www.nytimes.com/2016/01/02/us/politics/jeb-bush-presidential-campaign.html', 'published_date': '2016-01-01T16:35:09+0000', 'section': 'U.S.', 'subsection': 'Politics'}
{'title': 'Obama to Consider Executive Actions on Gun Violence', 'snippet': 'President Obama will meet with Attor

In [101]:
batch_size = 7
startDate = datetime(2024, 1, 1)
endDate = datetime(2024,11,23)


def getArticlesForDates(startdate, enddate):
    
    endpoint = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
    params = {
          "fq": 'subsection_name:("Politics" OR "Economic" OR "Business")',
          "begin_date": startdate.strftime("%Y%m%d"),
          "end_date": enddate.strftime("%Y%m%d"),
          "sort": "relevance", 
          "api-key": "MDhBwmCEr3FfhRNn63ogkPOowV3bklnY"
        }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        return response.json()
    
    else:
        print(f"Error: {response.status_code} - {response.text}")

totalDays = (endDate - startDate).days + 1  # Total number of days
totalsteps = (totalDays + batch_size - 1) // batch_size
articlesData = []
current_date = startDate
with tqdm(total=totalsteps, desc="Fetching Articles") as pbar:
    current_date = startDate
    while current_date <= endDate:
        range_end = min(current_date + timedelta(days=batch_size - 1), endDate)
        data = getArticlesForDates(current_date, range_end)
        if data:
            # Extract article details
            docs = data.get('response', {}).get('docs', [])
            for doc in docs:
                article_info = {
                    "title": doc.get("headline", {}).get("main", ""),
                    "snippet": doc.get("snippet", ""),
                    "url": doc.get("web_url", ""),
                    "published_date": doc.get("pub_date", ""),
                    "section": doc.get("section_name", ""),
                    "subsection": doc.get("subsection_name", ""),
                }
                articlesData.append(article_info)
        # Move to the next day and update the progress bar
        current_date += timedelta(days=batch_size)
        pbar.update(1)
        time.sleep(7) 

articledf = pd.DataFrame(articlesData)
pd.set_option('display.max_rows',500)
print(articledf)


Fetching Articles:  21%|██▏       | 10/47 [01:09<04:34,  7.43s/it]

Error: 429 - {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : 5868c784-93ae-406a-a6a9-898525807e3a","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


Fetching Articles:  23%|██▎       | 11/47 [01:16<04:26,  7.40s/it]

Error: 429 - {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : 5868c784-93ae-406a-a6a9-898525807e3a","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


Fetching Articles:  26%|██▌       | 12/47 [01:25<04:30,  7.74s/it]

...

Error: 429 - {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : 5868c784-93ae-406a-a6a9-898525807e3a","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


Fetching Articles:  94%|█████████▎| 44/47 [05:27<00:22,  7.39s/it]

Error: 429 - {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : 5868c784-93ae-406a-a6a9-898525807e3a","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


Fetching Articles: 100%|██████████| 47/47 [05:57<00:00,  7.61s/it]

                                                 title  \
0    Pressure Builds on Pentagon to Explain Timelin...   
1    With Time Running Short, Liz Cheney Implores R...   
2    From Lebanon to the Red Sea, a Broader Conflic...   
3    Japanese Company’s Bid for U.S. Steel Tests Bi...   
4    Trump Takes Aim at Haley as Primary Enters Fin...   
5    Michigan Republicans Hold Disputed Vote to Rem...   
6    On Jan. 6 Anniversary, Trump Repeats Lie That ...   
7    Defense Secretary Kept White House in the Dark...   
8    Biden to Deliver State of the Union Address on...   
9                     House Dysfunction by the Numbers   
10                          How the Iowa Caucuses Work   
11   Trump Has Made Claims About Caucus Fraud. What...   
12                 What to Expect at the Iowa Caucuses   
13   Democrats Fret That Biden’s Power Players Are ...   
14   Larry Hogan Backs Nikki Haley for G.O.P. Presi...   
15   7 Numbers That Tell the Tale of the G.O.P. Pri...   
16   How Colle

In [102]:
import pandas as pd
import os

# Assuming you have the articlesData as a DataFrame
articledf = pd.DataFrame(articlesData)

# Specify a valid file path in your working directory
output_path = os.path.join(os.getcwd(), "nyt_articles_2024.csv")

# Save the DataFrame to a CSV
articledf.to_csv(output_path, index=False)

# Confirm the path
print(f"File saved to: {output_path}")

File saved to: /Users/jadakethompson/Documents/Projects/CodeJam14/data/nyt_articles_2024.csv


In [104]:
import glob
csv_files = glob.glob(f"nyt_articles_20*.csv")
dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)

# Display the merged DataFrame
print(merged_df)
output_path = os.path.join(os.getcwd(), "nyt_articles_merged.csv")

# Save the DataFrame to a CSV
merged_df.to_csv(output_path, index=False)

                                                  title  \
0     An Ice Cream Mogul and His Fellow Burlingtonia...   
1     Bill Clinton, Back in Iowa, Refuses to Respond...   
2               Donald Trump Brings Gridlock to Vermont   
3     Ad Wars of 2016 Campaign Erupt in a Changing T...   
4     Ted Cruz’s Iowa Bus Tour: Using ‘Star Wars’ to...   
...                                                 ...   
3109  Trump Picks Key Figure in Project 2025 for Pow...   
3110  Pam Bondi, a $25,000 Donation and Trump Univer...   
3111                          Matt Gaetz and the D-word   
3112  Republicans Blocking Proposal to Narrow Part o...   
3113  Elon Musk Gets a Crash Course in How Trumpworl...   

                                                snippet  \
0     In a liberal enclave fiercely loyal to Bernie ...   
1     Bill Clinton returned to the campaign trail in...   
2     Donald J. Trump brought gridlock to Burlington...   
3     Candidates and the outside groups supporting t...